# Aligned Beethoven Sonata Data for chord-eval

In [23]:
from glob import glob
import logging
from pathlib import Path
from tqdm import tqdm

import harmonic_inference.data.piece as piece
from harmonic_inference.utils.eval_utils import get_labels_df

OUTPUT_DIR = Path("Beethoven-labels")
if not OUTPUT_DIR.exists():
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [24]:
from harmonic_inference.data.corpus_reading import load_clean_corpus_dfs

files_df, measures_df, chords_df, notes_df = load_clean_corpus_dfs('corpus_data')

In [4]:
file_ids = list(files_df.loc[files_df["corpus_name"] == "Beethoven-Sonatas"].index)
print(files_df.loc[148])
chords_beethoven = chords_df.loc[file_ids]

chords_beethoven.loc[chords_beethoven["changes"] == "7", ["label", "numeral", "chord_tones", "globalkey", "localkey"]]
chords_beethoven["changes"].value_counts().index

corpus_name    Beethoven-Sonatas
file_name               31-3.tsv
Name: 148, dtype: object


Index(['64', '4', '9', '2', 'b3', '+2', '6', 'b9', '11', 'b64', '#7', '94',
       '13', '974', '#4#2', '+4', 'b6', '+6', 'b5', '+b9', '134', '#6#4', '#4',
       '119', '7', '964', '#11', '#6', '#9', '42', '9#7', '74', '#964', 'b2',
       '116', '112', '#64', '#2', 'b94', '11#7', '#764', '#96#4', '97', '6#4',
       '^2', 'b4', '#74', '9#74', '72', '97b64', '4#2', 'b13', '96', '+b8',
       '#11#2', '#9#7', '#762', '4+2', '#9#74', 'b3+2', '9#764', 'b32', '9#54',
       'b964', '#72', '6#2', '#96', '14', '1197', '+#7', '11#762', '764', '#5',
       '7#2', '11#7b6', '+b2', '4b2'],
      dtype='object')

In [22]:
import harmonic_inference.utils.harmonic_utils as hu
from harmonic_inference.data.data_types import PitchType
from harmonic_inference.data.chord import Chord

import importlib
importlib.reload(hu)

count = 0
for index, row in chords_beethoven.loc[~chords_beethoven["changes"].isna()].iterrows():
    chord = Chord.from_series(row, measures_df.loc[index[0]], PitchType.TPC)
    
    changes = hu.get_added_and_removed_pitches(chord.root, chord.chord_type, chord.suspension, chord.key_tonic, chord.key_mode)

    print(row[["label", "globalkey", "localkey"]])
    print(
        "Changes:",
        changes,
        ", ".join(
            f"{hu.get_pitch_string(int(orig), PitchType.TPC) if orig != '' else ''}: {hu.get_pitch_string(int(new), PitchType.TPC)}"
            for orig, new in changes.items()
        )
    )

    count += 1
    if count >= 100:
        break

label        V(4)
globalkey       f
localkey        i
Name: (89, 6), dtype: object
Changes: {'19': '14'} E: F
label        ii6(2)
globalkey         f
localkey        III
Name: (89, 13), dtype: object
Changes: {'13': '15'} Bb: C
label        ii6(2)
globalkey         f
localkey        III
Name: (89, 18), dtype: object
Changes: {'13': '15'} Bb: C
label        ii6(2)
globalkey         f
localkey        III
Name: (89, 23), dtype: object
Changes: {'13': '15'} Bb: C
label        V7(+b9)
globalkey          f
localkey         III
Name: (89, 27), dtype: object
Changes: {'': '+7'} : Fb
label        V7(+b9)
globalkey          f
localkey         III
Name: (89, 29), dtype: object
Changes: {'': '+7'} : Fb
label        V7(+b9)]
globalkey           f
localkey          III
Name: (89, 31), dtype: object
Changes: {'': '+7'} : Fb
label        V(64)
globalkey        f
localkey       III
Name: (89, 49), dtype: object
Changes: {'13': '15', '16': '11'} Bb: C, G: Ab
label        V(64)
globalkey        f
localke

In [25]:
for fh_filename in tqdm(glob(str(Path("../functional-harmony/data/BPS/scores/*.mxl")))):
    music_xml_path = Path(fh_filename)
    label_csv_path = (
        music_xml_path.parent.parent /
        "chords" /
        Path(str(music_xml_path.stem) + ".csv")
    )

    if not label_csv_path.exists():
        logging.error(f"Label file {label_csv_path} does not exist. Skipping.")
        continue

    fh_score = piece.get_score_piece_from_music_xml(music_xml_path, label_csv_path)

    _, number, movement = music_xml_path.stem.split("_")

    dcml_corpus = "Beethoven-Sonatas"
    dcml_file_name = f"{number}-{movement[-1]}.tsv"

    df = files_df.loc[(files_df["corpus_name"] == dcml_corpus) & (files_df["file_name"] == dcml_file_name)]

    if len(df) == 0:
        logging.error(f"No matching df file_id found for score {music_xml_path}. Skipping.")
        continue

    if len(df) > 1:
        logging.error(f"Multiple matching df file_ids found for score {music_xml_path}: {df}.\nUsing the first.")

    file_id = df.index[0]
    
    try:
        dcml_score = piece.get_score_piece_from_data_frames(
            notes_df.loc[file_id],
            chords_df.loc[file_id],
            measures_df.loc[file_id],
            use_suspensions=True,
        )
    except KeyError:
        logging.error(f"No matching chord_df data found for score {music_xml_path} (file_id {file_id}). Skipping.")
        continue

    fh_label_df = get_labels_df(fh_score, tpc_c=0)
    fh_label_df.to_csv(OUTPUT_DIR / f"fh-{number}-{movement[-1]}.tsv", index=False, sep="\t")

    dcml_label_df = get_labels_df(dcml_score, tpc_c=0)
    dcml_label_df.to_csv(OUTPUT_DIR / f"dcml-{number}-{movement[-1]}.tsv", index=False, sep="\t")

  0%|          | 0/32 [00:07<?, ?it/s]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()